In [ ]:
import psycopg2
import nltk
import pandas as pd
import numpy as np
import pyodbc

In [ ]:
nltk.download('punkt')

In [ ]:
def execQuery(query):
    try:
        connection = psycopg2.connect(user = "postgres",
                                      password = "root",
                                      host = "localhost",
                                      port = "5432",
                                      database = "postgres")
        cursor = connection.cursor()
        cursor.execute(query)
        record = cursor.fetchall()
        return record
    except (Exception, psycopg2.Error) as error :
        connection = False
        print ("Error while connecting to PostgreSQL", error)
    finally:
        if(connection):
            cursor.close()
            connection.close()
            print("Executed query and closed connection.")

In [ ]:
articlesContent = execQuery("""Select content
from article""")

In [ ]:
#connection = psycopg2.connect(user = "postgres",
#                                      password = "root",
#                                      host = "localhost",
#                                      port = "5432",
#                                      database = "postgres")

# usage: specify file location, sample size and seed(used by random) 
filepath = '/home/daniel/OneDrive/KUuni/DataScience/Python/small.csv'
#filepath = 'news_sample.csv' # <- overwrite for setup
s = 250                    # desired sample size(seems to have slack ie. not exact)
seed = 1                     # seed used by Pseudorandom number generator

df = pd.read_csv(filepath, index_col = [0]).sample(n=s, random_state=seed)

In [ ]:
df_content = df['content'].str.lower()

In [ ]:
allWords = []

for content in df['content'].to_numpy():
    # np.unique(np.concatenate([allWords, nltk.word_tokenize(content[0])])) <- might to add or remove [0]
    allWords = np.unique(np.concatenate([allWords, nltk.word_tokenize(content)]))

len(allWords)

In [ ]:
series_content_tokens = df_content.apply(nltk.word_tokenize)

In [ ]:
series_content_tokens.explode().value_counts()

In [ ]:
series_content_tokens = series_content_tokens.explode()

In [ ]:
df_content_tokens = series_content_tokens.to_frame()
df_content_tokens = df_content_tokens.reset_index().rename(columns={'index':'content_index'})
series_content_tokens = df_content_tokens.groupby('content_index')['content'].value_counts(normalize=True)

In [245]:
series_content_tokens

content_index  content     
0              the             0.084715
               .               0.040516
               of              0.036832
               to              0.029466
               in              0.025783
                                 ...   
249            unacceptable    0.007752
               wa              0.007752
               wednesday       0.007752
               will            0.007752
               yangon          0.007752
Name: content, Length: 76994, dtype: float64

In [250]:
# create TF
TF = series_content_tokens.unstack(level=0)
TF.head()

content_index,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
content,,,,,,,,,,,,,,,,,,,,,
!,0.007366,0.008969,NaN,NaN,NaN,NaN,0.026316,NaN,0.003388,NaN,...,0.004484,NaN,NaN,NaN,NaN,0.001922,0.002985,NaN,0.002016,NaN
#,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
$,0.001842,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.014037,NaN,...,NaN,NaN,0.003247,NaN,NaN,0.001281,NaN,NaN,NaN,NaN
%,NaN,0.004484,NaN,NaN,NaN,NaN,NaN,NaN,0.003872,NaN,...,NaN,NaN,NaN,NaN,NaN,0.004484,NaN,NaN,NaN,NaN
&,NaN,0.004484,0.001912,NaN,NaN,NaN,NaN,NaN,0.000968,NaN,...,0.002242,NaN,NaN,NaN,NaN,0.000641,0.002985,NaN,NaN,NaN


In [259]:
TF[TF > 0] / (1 + len(DF))

NameError: name 'log' is not defined

In [258]:
len(TF)

17928

In [ ]:
wordsDict = {}

for word in df_content_tokens['content']:
    test = 0
    for content in df_content:
        if word in content[0]:
            test += 1

    wordsDict[word] = np.log(250/(test + 1))
    test = 0
len(wordsDict)